In [1]:
import googlemaps
import os
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from typing import Literal


In [ ]:
os.environ["GOOGLE_API_KEY"] = ""
os.environ["GPLACES_API_KEY"] = ""
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"
api = os.environ["GPLACES_API_KEY"]
gmaps = googlemaps.Client(api)
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, max_retries = 3)

In [3]:
# Initializing the Base Class to ensure correct formatting of model
class DistanceToolFormatting(BaseModel):
    origins: str = Field(description = "The starting point from where we want to see")
    destinations: str = Field(description = "The Ending Point which is used to calculate the distance")
    mode: Literal["walking", "driving", "transit"] = Field(default = "walking", description = "The mode which is used to calculate the duration")

In [4]:
# Tool Calling for distance matrix calculation
@tool("distance_matrix", description = "This tool is used to measure the distance between two locations", args_schema = DistanceToolFormatting)
def distance_measurement_tool(origins: str, destinations: str, mode: str = "walking"):
    """ This function is used to measure the distance between two locations. 
        The locations should be in str and the origins is the location person want to go from and destinations is the location person want to go to
        The mode is the mode of travel, it can be "walking", "driving" or "transit" 
        The return or outputs of this tools is string which contain the distance and time taken. 
    """
    result = gmaps.distance_matrix(origins, destinations, mode = mode)
    print(f"The result from distance tool: {result}")
    distances = []
    durations = []

    for element in result['rows']:
        for status in element['elements']:
            if status['status'] == "ZERO_RESULTS":
                return (
                    "No ground route found between the selected locations. "
                    "This route may cross restricted borders or require air travel."
                )
    for row in result['rows']:
        for sub_row in row["elements"]:
            distance = sub_row['distance']['text']
            duration = sub_row['duration']['text']
            distances.append(distance)
            durations.append(duration)
    final_formatted_result = f"The duration between {origins} and {destinations} is {durations} min and distance is {distances} km"
    return final_formatted_result

In [8]:
distance_prompt = """You are DistanceToolAgent. Your job is to calculate the distance and estimated time between two locations using the distance_matrix tool.

TOOLS:
- distance_measurement_tool(origins, destinations, mode): Returns the distance and time between two locations.

USAGE:
- "from [origin] to [destination]" → calculate distance and time for the specified route
- "distance between [origin] and [destination] by [mode]" → calculate distance and time with a specified mode of travel ("walking" or "driving")

RULES:
1. Only use tool outputs, never fabricate data.
2. If tool returns an error such as "ZERO_RESULTS", report: "No ground route found between the selected locations. This route may cross restricted borders or require air travel."
3. Present the results in the following format:
   - "The duration between [origin] and [destination] is [duration] and distance is [distance]."
4. Handle both the distance and duration results appropriately. If multiple results exist, present them all.

FORMAT:
The result should be in the format:
"The [mode] duration between [origins] and [destinations] is [duration] and distance is [distance]."

EXAMPLES:
- "What is the distance and time from New York to Los Angeles by driving?"
- "How long does it take to walk from Paris to Berlin?"

RULES FOR SPECIAL CASES:
- If there are multiple results, return the best match.
- If the distance_matrix tool returns no valid results, respond with a message stating no route is found.
"""



In [ ]:
distance_agent = create_agent(model = model, tools = [distance_measurement_tool], system_prompt = distance_prompt)
distance_agent.invoke({"messages": "Tell me distance and time from lahore to islamabad, both walking, driving and transit"})

The result from distance tool: {'destination_addresses': ['Islamabad, Pakistan'], 'origin_addresses': ['Lahore, Pakistan'], 'rows': [{'elements': [{'distance': {'text': '295 km', 'value': 295044}, 'duration': {'text': '2 days 19 hours', 'value': 240794}, 'status': 'OK'}]}], 'status': 'OK'}
The result from distance tool: {'destination_addresses': ['Islamabad, Pakistan'], 'origin_addresses': ['Lahore, Pakistan'], 'rows': [{'elements': [{'distance': {'text': '373 km', 'value': 372810}, 'duration': {'text': '5 hours 15 mins', 'value': 18900}, 'status': 'OK'}]}], 'status': 'OK'}
The result from distance tool: {'destination_addresses': ['Islamabad, Pakistan'], 'origin_addresses': ['Lahore, Pakistan'], 'rows': [{'elements': [{'distance': {'text': '378 km', 'value': 378277}, 'duration': {'text': '4 hours 31 mins', 'value': 16284}, 'status': 'OK'}]}], 'status': 'OK'}


{'messages': [HumanMessage(content='Tell me distance and time from lahore to islamabad, both walking, driving and transit', additional_kwargs={}, response_metadata={}, id='a73a8f68-1b20-4129-98be-37ddc4bc8365'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'distance_matrix', 'arguments': '{"destinations": "islamabad", "mode": "transit", "origins": "lahore"}'}, '__gemini_function_call_thought_signatures__': {'968bee64-7d03-493c-a738-2fcb542f5925': 'CvkBAXLI2nzcsnFnkfXWo5pfV7/CiZ7pslbBWBoDkeQSgm+M+URw+DwW9qNjgNASLglOOmFF8qZ4iTbxuLOOi+zdgFPDJwPtie5dIvV2CfuEKyBdBZyC/FCWExvOtkc0Nzea+up+fC6YodtZITFWTSiST+tq1W0IipA28gEZUPwkxD9zdntCsafckDTDtzfLkk6FrhP+CUdFr2aotc3XVynLtCB+bul0wv/30igQ3/HT8fkYjXWTXzZm/87oE4ZO82OX24SumTJDzDMIZJIhL/Zd04cCWjev3LfNO5pDasQuvvekd7AW/YyZq40LtpaxbdnpWNku02mOwbne'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b9ce9-09dd-7f02-ab17-4ad733f750a7